In [ ]:
###################### BUSINESS ENDPOINT FUNCTIONS

In [ ]:
url = 'https://api.yelp.com/v3/businesses/search'
client_id = 'SrP-vVinyIiYU2-NOuqnOA'
api_key = 'CaJ2qY7XSMjW7nhV9IK8KiEYI6Yj3kPAXR6Jdgstzo-YOFuAfT3AbrRF6UID6P9ia75sS8dzlih1HNcZEK1EHzu8Dp-gOnC3eVdyNpHKt4iJIUIOC7ovxCM68jrpXXYx'
headers = {
        'Authorization': 'Bearer {}'.format(api_key),
        }
        # what type of business do you want to search
term = 'bagel'

        #where do you want to perform this search
location = 'East Village'

        # what is your other parameter you want to search against
radius = 5000

limit = 50
    
offset = cur
    
url_params = {
                "term": term.replace(' ', '+'),
                "location": location.replace(' ', '+'),
                "radius" : radius,
                "limit" : limit,
                "offset" : offset
        
                }

In [6]:
# API Call Function
def api_call(url, headers, url_params):
    response = requests.get(url, headers=headers, params=url_params)
    data = response.json()
    business = data['businesses']
    return business

In [5]:
# Data Parsing Function

def parse_data(data_list):
    parsed_data = []  
    for business in data_list:
        try:
            biz_tuple = (business['id'], business['name'], ' '.join(business['location']['display_address']), business['rating'], len(business['price']), business['review_count'])
            parsed_data.append(biz_tuple)    
        except:
            biz_tuple = (business['id'], business['name'], ' '.join(business['location']['display_address']), business['rating'], 0, business['review_count'])
            parsed_data.append(biz_tuple)
    return parsed_data

In [ ]:
# Insert into DB Function

def insert_func(cnx, cursor, parsed_data):
    stmt = "INSERT INTO businesses (business_id, name, address, rating, price, review_count) VALUES (%s, %s, %s, %s, %s, %s)"
    cursor.executemany(stmt, parsed_data)
    cnx.commit()
    print('Check the workbench.')

In [7]:
# Script/Function that combines the three functions above into a single process.
def paginate(url_params, headers, url):
    cur = 0
    while cur < 1000:
        url_params['offset'] = cur
        businesses = api_call(url, headers, url_params)
        parsed_data = parse_data(businesses)
        insert_func(cnx, cursor, parsed_data)
        cur += 50   

In [ ]:
##############################################################################################

In [ ]:
#################################### REVIEWS #################################################

In [ ]:
############################ REVIEW ENDPOINT FUNCTIONS

In [7]:
cursor.execute("""SELECT business_id FROM yelp.businesses;""")


NameError: name 'cursor' is not defined

In [ ]:
working_list = cursor.fetchall()
biz_id_list =[]
for b in working_list:
    for x in b:
        biz_id_list.append(x)

In [ ]:
biz_id_list

In [4]:
# API Call Function
def rev_api_call(url, headers):
    response = requests.get(url, headers=headers)
    data = response.json()
    return data['reviews']

In [5]:
# Data Parsing Function
def rev_parse_data(reviews, business_id):
    parsed_revs = []
    for review in reviews:
        rev_tuple = (review['id'], review['review_text'], review['rating'], review['time_created'], business_id)
        parsed_revs.append(rev_tuple)
    return parsed_revs

In [6]:
# Insert into DB Function
def rev_insert_func(cnx, cursor, parsed_data):
    stmt = "INSERT INTO reviews (review_id, review_text, rating, time_created, business_id) VALUES (%s, %s, %s, %s, %s)"
    cursor.executemany(stmt, parsed_data)
    cnx.commit()
    print('Check the workbench.')

In [ ]:
# Script/Function that combines the three functions above into a single process.
def rev_paginate(business_id_list):
    for b in business_id_list:
        url= f'(https://api.yelp.com/v3/businesses/{b}/reviews)'
        reviews = api_call(url, headers)
        parsed_reviews = parse_rev_data(reviews, {b})
        insert_func(cnx, cursor, parsed_reviews)